# Protein Fold Dataset Creator Dmeo

This Demo is a simple example of using Dataset operations to create a datset

## Imports

In [1]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import col, when
from mmtfPyspark.ml import proteinSequenceEncoder
from mmtfPyspark.mappers import structureToPolymerChains
from mmtfPyspark.webfilters import blastCluster
from mmtfPyspark.filters import containsLProteinChain
from mmtfPyspark.datasets import secondaryStructureExtractor
from mmtfPyspark.webfilters import pisces
from mmtfPyspark.io import MmtfReader

## Define addProteinFoldType Class

In [2]:
def addProteinFoldType(data, minThreshold, maxThreshold):
    '''
    Adds a column "foldType" with three major secondary structure class:
    "alpha", "beta", "alpha+beta", and "other" based upon the fraction of alpha/beta content.

    The simplified syntax used in this method relies on two imports:
        from pyspark.sql.functions import when
        from pyspark.sql.functions import col

    Attributes:
        data (Dataset<Row>): input dataset with alpha, beta composition
        minThreshold (float): below this threshold, the secondary structure is ignored
        maxThreshold (float): above this threshold, the secondary structure is ignored
    '''

    return data.withColumn("foldType", \
                           when((col("alpha") > maxThreshold) & (col("beta") < minThreshold), "alpha"). \
                           when((col("beta") > maxThreshold) & (col("alpha") < minThreshold), "beta"). \
                           when((col("alpha") > maxThreshold) & (col("beta") < minThreshold), "alpha+beta"). \
                           otherwise("other")\
                           )

## Configure Spark Context

In [3]:
conf = SparkConf() \
            .setMaster("local[*]") \
            .setAppName("ProteinFoldDatasetCreatorDemo")

sc = SparkContext(conf = conf)

## Read MMTF Hadoop sequence file

Create non-redundant set (<=40% seq. identity) if L-protein chains

In [4]:
path = "../../resources/mmtf_reduced_sample/"
sequenceIdentity = 40

pdb = MmtfReader \
        .readSequenceFile(path, sc) \
        .filter(blastCluster(sequenceIdentity)) \
        .flatMap(structureToPolymerChains()) \
        .filter(blastCluster(sequenceIdentity)) \
        .filter(containsLProteinChain())

## Get secondary structure content

In [5]:
data = secondaryStructureExtractor.getDataset(pdb)

## Classify chains by secondary structure type

In [6]:
minThreshold = 0.05
maxThreshold = 0.15
data = addProteinFoldType(data, minThreshold, maxThreshold)

## Add Word2Vec encoded feature vector

In [7]:
encoder = proteinSequenceEncoder(data)
n = 2 # Create 2-grams
windowSize = 25 # 25-amino residue window size for Word2Vec
vectorSize = 50 # dimension of feature vector
data = encoder.overlappingNgramWord2VecEncode(n, windowSize, vectorSize).cache()

data.printSchema()
data.show(10)

root
 |-- structureChainId: string (nullable = false)
 |-- sequence: string (nullable = false)
 |-- alpha: float (nullable = false)
 |-- beta: float (nullable = false)
 |-- coil: float (nullable = false)
 |-- dsspQ8Code: string (nullable = false)
 |-- dsspQ3Code: string (nullable = false)
 |-- foldType: string (nullable = false)
 |-- ngram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)

+----------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------+--------------------+--------------------+
|structureChainId|            sequence|      alpha|      beta|      coil|          dsspQ8Code|          dsspQ3Code|foldType|               ngram|            features|
+----------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------+--------------------+--------------------+
|          1FDK.A|ALWQFNGMIKCKIPSSE...|  0.

## Keep only a subset of relevant fields for futher processing

In [8]:
data = data.select("structureChainId", "alpha", "beta", "coil", "foldType", "features")

data.show(10)

+----------------+-----------+----------+----------+--------+--------------------+
|structureChainId|      alpha|      beta|      coil|foldType|            features|
+----------------+-----------+----------+----------+--------+--------------------+
|          1FDK.A|  0.4796748|0.08943089|0.43089432|   other|[0.33268375846473...|
|          1FDL.L|0.060747664| 0.5046729|0.43457943|   other|[0.18110659749448...|
|          1FDL.H|0.055045873| 0.5091743| 0.4357798|   other|[-0.0615457752140...|
|          1FDL.Y| 0.41860464|0.11627907| 0.4651163|   other|[0.21163045239518...|
|          1FDM.A|        0.5|       0.0|       0.5|   alpha|[-0.1784528120401...|
|          1FDN.A| 0.14545454|0.21818182| 0.6363636|   other|[-0.2569646641474...|
|          1FDO.A| 0.37062937|0.17202798|0.45734265|   other|[-0.1180968281287...|
|          1FDP.A|0.108597286| 0.3484163|0.54298645|   other|[-0.3794333174601...|
|          1FDP.B| 0.09950249|0.38308457|0.51741296|   other|[-0.3794333174601...|
|   

## Terminate Spark Context

In [9]:
sc.stop()